### Perform Necessary Installations

In [ ]:
!pip install bs4
!pip install requests
!pip install pandas
!pip numpy

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import numpy as np

### Define Helper Functions
We will use those throughout our notebook as they make our code resuable, extensible, and readable 

In [ ]:
def flatten_comprehension(matrix):
     return [item for row in matrix for item in row]


def extract_rows_from_tables(tables):
    rows = []
# Now, iterate over each table in the ResultSet
    for table in tables:
        # For each table, you can now call find_all on it
        rows.append(table.find_all('tr'))
        # Do something with rows, e.g., print them
        
    
    return rows

def extract_raw_data(column_data,recurse_thru_a = False, find_tags = 'td', extract_provider_links = False, pattern =''):
    raw_data = []
    for row in column_data:
        row_data = row.find_all(find_tags)
        individual_row_data = []
        
        if(recurse_thru_a == True):
            extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
#             print("extracted links: ", extracted_links)
            link_to_page = extracted_links[0] if extracted_links else None
#             print(f'Link to page: {link_to_page}')
            individual_row_data.append(link_to_page)
        
        individual_row_data +=[data.text.strip() for data in row_data]
        
        if extract_provider_links == True:
            extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
            for element in extracted_links:
                matches = re.findall(pattern, element)
                if(matches):
                    individual_row_data.append(element)
#                 else:
#                     individual_row_data.append('')
        
#                 print(f"'{element}' contains {len(matches)} occurrence(s) of {pattern}")
                
#         individual_row_data +=[data.text.strip() for data in row_data]
#         print(f'{individual_row_data}')
        raw_data.append(individual_row_data)
    return raw_data
def extract_providers_link(column_data, find_tags = 'td',  pattern = ''):
    raw_data = []
    for row in column_data:
        row_data = row.find_all(find_tags)
        individual_row_data = []
        
       
        extracted_links = [data.find('a')['href'] if data.find('a') else data.text.strip() for data in row_data]
        
        
#         print(extracted_links)
        for element in extracted_links:
            matches = re.findall(pattern, element)
            if(matches):
                # print(f"'{element}' contains {len(matches)} occurrence(s) of {pattern}")
                individual_row_data +=[data.text.strip() for data in row_data]
                individual_row_data.append(element)
                raw_data.append(individual_row_data)

                
#         individual_row_data +=[data.text.strip() for data in row_data]
#         print(f'{individual_row_data}')
    return raw_data
def append_data_to_df(df, raw_data):
    for data in raw_data:
#         print(f'data: {data}')
        length = len(df)
        df.loc[length] = data
    return df

def preprocess_extracted_sats(cont_sats_extracted, row_names, attr_index = 0):
    for index, sat in enumerate(cont_sats_extracted):
#         print("len(sat) = > ",len(sat))
#         print("len(row_names) = > ",len(sat))
        
        if len(sat) <= len(row_names) - 1:
#             print(f'found a channel "{sat}"')
            sat_dir = cont_sats_extracted[index - 1][attr_index]
#             print(f'New Attr "{sat_dir}"')
            sat.insert(attr_index, sat_dir)
            cont_sats_extracted[index] = sat
#             print(f'New channel "{cont_sats_extracted[index]}"')

    return cont_sats_extracted

def append_region_to_extracted_sats(cont_sats_extracted, continent):
    for index, sat in enumerate(cont_sats_extracted):
        cont_sats_extracted[index] =  sat + [continent]
    return cont_sats_extracted

## Extract Satellites for All Regions

### List Continents and concatenate with their url

In [ ]:
continents = ['asia', 'europe', 'atlantic', 'america']
baseurl = 'https://www.lyngsat.com/'

# Construct the URLs
endpoints = []
for c in continents:
    endpoints.append(f'{baseurl}{c}.html')
endpoints

### Scrape each Endpoint

In [ ]:
satellites = []
for index, c in enumerate(continents):
    page = requests.get(endpoints[index])
    soup = bs(page.text, 'html')
    
    
    satellites.append({f'{c}':soup})

In [ ]:
row_names = ['Position','Name','Frequency','Launch_Date', 'Region']
cont_sats_df = pd.DataFrame(columns = row_names)
# 
for index, cont in enumerate(continents):
#     print('continent: ',cont)
    cont_sats_raw = satellites[index][cont]
    table = cont_sats_raw.find_all('table')[11]
    
    column_data = table.find_all('tr')
    
    cont_sats_extracted = extract_raw_data(column_data)
    
    cont_sats_extracted = preprocess_extracted_sats(cont_sats_extracted, row_names)
    cont_sats_extracted = append_region_to_extracted_sats(cont_sats_extracted, cont)    

    cont_sats_df = append_data_to_df(cont_sats_df, cont_sats_extracted)
    
# Switch the first (position) with the second column (name)
cont_sats_df = cont_sats_df.iloc[:, [1, 0] + list(range(2, len(cont_sats_df.columns)))]


In [ ]:
cont_sats_df

### Get Launching Details

In [ ]:
launch_endpoint = baseurl + "launches/index.html"

page = requests.get(launch_endpoint)
soup = bs(page.text, 'html')



In [ ]:
table = soup.find_all('table')[15]
column_data = table.find_all('tr')

sats_extracted = extract_raw_data(column_data)

sats_extracted = [[entry[2], entry[3]] for entry in sats_extracted]
row_names = ['Sat_Name', 'Rocket']
rockets_df = pd.DataFrame(columns = row_names)

rockets_df = append_data_to_df(rockets_df, sats_extracted)
# rockets_df

merged_df = pd.merge(cont_sats_df, rockets_df, left_on = 'Name', right_on = 'Sat_Name', how='left')
# merged_df = merged_df.fillna(None)

merged_df

In [ ]:
cont_sats_df[['Position_Longitude', 'Position_Direction']] = cont_sats_df['Position'].str.split('\u00b0', expand=True)
cont_sats_df.drop('Position', axis=1, inplace=True)

cont_sats_df


In [ ]:
cont_sats_df.to_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/Satellites.csv',index=False)

In [ ]:
continents = ['asia', 'europe', 'atlantic', 'america']
baseurl = 'https://www.lyngsat.com/tracker/'

# Construct the URLs
endpoints = []
for c in continents:
    endpoints.append(f'{baseurl}{c}.html')
endpoints

In [ ]:
sat_rockets = []
for index, c in enumerate(continents):
    page = requests.get(endpoints[index])
    soup = bs(page.text, 'html')
    
    
    sat_rockets.append({f'{c}':soup})

In [ ]:
sat_rockets

In [ ]:
row_names = ['URL','Position','Name','Frequency', 'Launch Date', 'Region']
cont_sats_df = pd.DataFrame(columns = row_names)

for index, cont in enumerate(continents):

    cont_sats_rockets = sat_rockets[index][cont]
# cont_sats_rockets = sat_rockets[2]['atlantic']

#     print(cont_sats_rockets)

    table = cont_sats_rockets.find_all('table')[11]
#     print(table)
    
    column_data = table.find_all('tr')
    
#     print(column_data)
    
    cont_sats_extracted = extract_raw_data(column_data, True)

#     cont_sats_extracted = preprocess_extracted_sats(cont_sats_extracted, row_names)
    cont_sats_extracted = append_region_to_extracted_sats(cont_sats_extracted, cont)    
    
    cont_sats_df = append_data_to_df(cont_sats_df, cont_sats_extracted)
    
    
#     print(cont_sats_df)
# Switch the first (position) with the second column (name)
# cont_sats_df = cont_sats_df.iloc[:, [1, 0] + list(range(2, len(cont_sats_df.columns)))]
cont_sats_df


In [ ]:
cont_sats_df


In [ ]:
cont_sats_df['URL']

In [ ]:
rocket_list = []
for index, cont in enumerate(cont_sats_df['URL']):
    url = cont_sats_df['URL'][index]
    page = requests.get(url)
    soup = bs(page.text, 'html')
#     soup.find_all('font')[13]

    entry  = soup.find_all('font')
    
    entry = ' '.join(map(str,entry))

    x = re.search("launched with (.+) \d\d\d\d", entry)
#     print(x.group(1))
    
    sat_name = cont_sats_df['Name'][index]
    
    print(f'Satellite name: {sat_name}')
    if(x):
#         print(f'Found l')
        rocket_list.append(
            {
            f'{sat_name}': x.group(1)
        })
    else:
        print(f'Couldnt find a launching rocket for {sat_name} with entry {entry}')
        
    print(f'Processed {index + 1} Satellites out of {len(cont_sats_df.index)}')
    

In [ ]:
rocket_list

In [ ]:
rocket_list


list_of_lists = [list(d.items())[0] for d in rocket_list]
# new_list
row_names = ['Sat_name', 'L_Rocket']

rockets_df = pd.DataFrame(list_of_lists,columns = row_names)
list_of_lists

# new_rockets_df

In [ ]:
sat_df = pd.read_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/Satellites.csv')
# rockets_df = pd.read_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/Satellites_Rockets.csv')

# merged_df = pd.merge(sat_df, rockets_df, left_on = 'Name', right_on = 'Sat_name', how = 'left')
# merged_df = merged_df.drop(columns=['Sat_name'])
merged_df = pd.read_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/Satellites_Rockets.csv')

## Scrape Channels and Providers
For each satellite, we:
1. Scrape the Channels/ Providers records, treating them as one.
    a. Identify the Providers and merge with the original dataframe
    b. Save that
2. Assign Providers to Channels
3. Clean the Dataframes
    a. Split the System/SR/FEC columns
    b. Extract the languages
    c. Extract the Frequency

We begin by defining *helper functions* & *attributes*

In [ ]:
# Retrieves data tables from a given url
def get_table(url):
    
    page = requests.get(url)
    soup = bs(page.text, 'html')

    table = soup.find_all('table',{'border':"", 'cellpadding':"0", 'cellspacing':"0", 'width':"720"} )

    return table


In [ ]:
baseurl = 'https://www.lyngsat.com/'
# Contains the names of our Satellites
merged_df = pd.read_csv('E:/AUC/23-24/Spring/Database/Project/CSV Files/Satellites_Rockets.csv')

In [ ]:

# Prepare satellite names for incorporation into URLs

raw_sat_names = [link.replace(" ", "-") for link in merged_df['Name']]
raw_sat_names = [link.replace("'", "" ) for link in raw_sat_names]
raw_sat_names = [link.replace("ü", "u" ) for link in raw_sat_names]
raw_sat_names = [link.replace("/", "-" ) for link in raw_sat_names]
raw_sat_names = [link.replace("Ä", "A" ) for link in raw_sat_names]

sat_url_names = [re.split('-\(',link)[0] for link in raw_sat_names]
# baseurl

In [ ]:
sat_url_names

### Scrape Channels & Providers

In [ ]:
## SCRAPE CHANNELS + PROVIDERS

for index, sat_name in enumerate(sat_url_names):

    url = f'{baseurl}{sat_name}.html'
    print(f'{index + 1} Processing satellite {sat_name} with url {url}')

    # Scrape endpoint, retrieving all data tables 
    sat_table = get_table(url)

    # Retrieve all tr's from the returned tables
    # Returns a list of lists (/table)
    column_data = extract_rows_from_tables(sat_table)

    for i, table in enumerate(column_data):
    #     print("table#",i)
        if(len(table) > 2):
            _temp = column_data[i]
            _temp = _temp[2:len(_temp)-1]
        #     print(_temp)
            column_data[i] = _temp

    # Flattens the list. Now, we have a list of tr tags
    column_data = flatten_comprehension(column_data)
   
    # Clean and extract the data values from the tags
    channels_extracted = extract_raw_data(column_data)
    networks_extracted = extract_providers_link(column_data, pattern='.*providers')
    


    # Define the columns for our Main Dataframe
    row_names = ['Freq/beam','SR/FEC', 'SID', 'Provider/Channel','undef','Compression','VPID','Audio', 'Encryption', 'Src_Updated']
    
    # Define the columns for our Networks Dataframe
    netw_row_names = ['Freq/beam','SR/FEC', 'SID', 'Provider/Channel','undef','Compression','VPID','Audio', 'Encryption', 'Src_Updated', 'Provider_URL']
    
    # Construct the Network Dataframe
    netw_df = pd.DataFrame(networks_extracted, columns = netw_row_names)
    
    # Add Frequency/Beam to our channels (as inherited from the preceeding element)
    channels_extracted = preprocess_extracted_sats(channels_extracted, row_names)

    # Add System/SR/FEC to our channels (as inherited from the preceeding element)
    channels_extracted = preprocess_extracted_sats(channels_extracted, row_names, 1)

    # Construct the Main Dataframe
    sat_df = pd.DataFrame(channels_extracted, columns = row_names)    
    
    # Refine the Networks Dataframe to remove clutter/ redundant attributes
    netw_df = netw_df[['Provider/Channel', 'Provider_URL']]

    # Merge the Main and Network Dataframes
    merged_df = pd.merge(left=sat_df, right=netw_df, how='outer',left_on='Provider/Channel', right_on='Provider/Channel')

    # Adjust the filename to be safe (for saving the file on Windows)
    safe_sat_name = sat_name.replace('/', '-') # Replace '/' with '_'

    # Save the merged Dataframe
    merged_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v2/{index + 1}_{safe_sat_name}_channels.csv',index=False)



### Assign Providers to Channels

In [ ]:

for index, sat_name in enumerate(sat_url_names):
#     index = 1
#     sat_name = 'Intelsat-18'
    safe_sat_name = sat_name.replace('/', '-') # Replace '/' with '_'
    my_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v2/{index + 1}_{safe_sat_name}_channels.csv')

    # if len(my_df.index) == 0:
    #     continue
    print(f'{index + 1} Processing sat: {safe_sat_name}')

    my_df['is_Provider'] = np.where(my_df['Provider_URL'].astype(str).str.contains('http', regex=True, na=False), True, False)

    providers_df = my_df[my_df['is_Provider'] == True]
    channels_df = my_df[my_df['is_Provider'] == False]

    my_df['Provider'] = None


    for i in range(len(my_df.index)):
        isProvider = my_df.loc[i, 'is_Provider']
    
        if isProvider == True:
            my_df.loc[i, 'Provider'] = my_df.loc[i, 'Provider/Channel']
            continue

        if i > 0:
            my_df.loc[i, 'Provider'] = my_df.loc[i-1, 'Provider'] 
    my_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v2/assigned/{index + 1}_{safe_sat_name}_channels.csv',index=False)

my_df

### Clean our Dataframes

##### Define Helper Functions

In [ ]:


# Function to extract string until the last capital character
def extract_until_last_capital(s):
    match = re.search(r'(.*[A-Z])', s)
    if match:
        return match.group(1)
    return s
def find_pattern_and_join(entry):
    matches = re.findall(r"([A-Z][a-z]+)", entry)
    return ' '.join(matches) # Join the matches into a single string
def split_beam_eirp(entry, beam_eirp_index = 0):
    temp = re.split(r'(?<=\d)(?=\D)', entry)
    if beam_eirp_index == 1:
        # Check if the value is not None
        if(len(temp) > 1):
            return temp[beam_eirp_index] 
        else:
            'None'
    else:
        return temp[beam_eirp_index]

In [ ]:
skipped_sats_url = []

#### Perform The Cleaning

In [ ]:


for index, sat_name in enumerate(sat_url_names):
    safe_sat_name = sat_name.replace('/', '-') # Replace '/' with '_'
    hor_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v2/assigned/{index + 1}_{safe_sat_name}_channels.csv')

    print(f'{index + 1} Processing sat: {safe_sat_name}')

    if len(hor_df.index) == 0:
        url = f'{baseurl}{sat_name}.html'
        skipped_sats_url.append(url)
        hor_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v2/assigned/cleaned/{index+1}_{safe_sat_name}_channels.csv', index=False)

        continue
    
    # Drop irrelevant columns
    hor_df.drop(columns=['undef','Src_Updated'], inplace = True)  

    try:
        hor_df['FEC'] = hor_df['SR/FEC'].str[-3:]
        hor_df['SR'] = hor_df['SR/FEC'].str[-8:-3].str.extract('(\d+)')
        hor_df['SYSTEM'] = hor_df['SR/FEC'].str[0:6]
    except:
        print("PROBLEM YO")

    # Drop the now-old composite column
    hor_df.drop('SR/FEC', axis=1, inplace=True)

    # Extract Languages
    hor_df['Audio'] = hor_df['Audio'].astype(str)

    hor_df['Languages'] = hor_df['Audio'].apply(find_pattern_and_join)
    
    hor_df.drop('Audio', axis=1, inplace=True)
    
    # Split the Composite Frequency column    
    try:
#         extract_until_last_capital
        hor_df['Freq'] = hor_df['Freq/beam'].str[0:7].apply(extract_until_last_capital)
    except:
        print("PROBLEMSSSSSS")
    
    hor_df.to_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v2/assigned/cleaned/{index+1}_{safe_sat_name}_channels.csv', index=False)    

In [ ]:
# View the URLs that were skipped from cleaning due to lack of data
skipped_sats_url

In [ ]:
# View the current output for a cleaned dataframe
index = 1
safe_sat_name = 'Intelsat-18'

my_df = pd.read_csv(f'E:/AUC/23-24/Spring/Database/Project/CSV Files/providers/v2/assigned/cleaned/{index+1}_{safe_sat_name}_channels.csv')


In [ ]:
my_df.head()